# Baixando dados realizados pela Lei de Acesso à Informação (LAI) de Pedidos e Recursos/Reclamações.

Nesse notebook vamos apresentar como podemos, utilizando a linguagem de programação python, baixar de forma mais automática dos dados da lai de pedidos e recursos/reclamações. 

![tela fala br download](imagens/falabr_tela_download.png "Tela do Fala.Br para download dos arquivos de Pedidos e Recursos sem o texto do pedido.")

------------------------------
-- CAMPOS: PEDIDOS
------------------------------
- IdPedido - inteiro: identificador único do pedido (não mostrado no sistema);
- ProtocoloPedido - texto(17): número do protocolo do pedido;
- Esfera - texto(30): esfera do órgão destinatário do pedido;
- UF - texto(2): Sigla da UF do pedido quando não federal; 
- Municipio - texto(200): Nome do município do pedido quando não federal;
- OrgaoDestinatario  - texto(250): nome do órgão destinatário do pedido;
- Situacao - texto(200): descrição da situação do pedido;
- DataRegistro - Data DD/MM/AAAA HH:MM:SS : data de abertura do pedido;
- PrazoAtendimento - Data DD/MM/AAAA HH:MM:ss : data limite para atendimento ao pedido;
- FoiProrrogado - texto(3) "Sim" ou "Não" : informa se houve prorrogação do prazo do pedido;
- FoiReencaminhado - texto(3) "Sim" ou "Não": informa se o pedido foi reencaminhado;
- FormaResposta - texto(200): tipo de resposta escolhida pelo solicitante na abertura do pedido;
- OrigemSolicitacao - texto(50): informa se o pedido foi aberto em um Balcão SIC ou pela Internet;
- IdSolicitante - inteiro: identificador único do solicitante (não mostrado no sistema);
- AssuntoPedido - texto(200) : assunto do pedido atribuído pel SIC;
- SubAssuntoPedido - texto(200) : subassunto do pedido atribuída pelo SIC;
- Tag - texto(1024): as tags são marcadores no pedido para realizar classificações que não estão em assuntos/ subassuntos;
- DataResposta - Data DD/MM/AAAA HH:MM:SS : data da resposta ao pedido (campo em branco para pedidos que ainda estejam na situação "Em Tramitação");
- Decisao - texto(100) : tipo resposta dada ao pedido (campo em branco para pedidos que ainda estejam na situação "Em Tramitação");
- EspecificacaoDecisao - texto(200): subtipo da resposta dada ao pedido (campo em branco para pedidos que ainda estejam na situação "Em Tramitação");

------------------------------
-- CAMPOS: RECURSOS
------------------------------

- IdRecurso - inteiro: identificador único do recurso (não mostrado no sistema);
- IdRecursoPrecedente - inteiro: identificador único do recurso que precedeu este (não mostrado no sistema e em branco no caso de Recursos de 1ª Instância e Reclamações);
- IdPedido - inteiro: identificador único do pedido ao qual o recurso pertence (não mostrado no sistema);
- IdSolicitante - inteiro: identificador único do solicitante (não mostrado no sistema);
- ProtocoloPedido - texto(17): número do protocolo do pedido ao qual o recurso pertence;
- Esfera - texto(30): descrição da esfera do recurso;
- UF - texto(2): Sigla da UF do recurso quando não federal;
- Municipio - texto(200): Nome do município do recurso quando não federal;
- OrgaoDestinatario - texto(250): nome do órgão destinatário do recurso;
- Instancia - texto(80): descrição da instância do recurso;
- Situacao - texto(80): descrição da situação do recurso;
- DataRegistro - Data DD/MM/AAAA HH:MM:SS : data de abertura do recurso;
- PrazoAtendimento - Data DD/MM/AAAA HH:MM:SS : data limite para atendimento ao recurso;
- OrigemSolicitacao - texto(50): informa se o recurso foi aberto em um Balcão SIC ou pela Internet;
- TipoRecurso - texto(80): motivo de abertura do recurso;
- DataResposta - Data DD/MM/AAAA HH:MM:SS : data da resposta ao recurso (campo em branco para recursos que ainda estejam na situação "Em Tramitação");
- TipoResposta - texto(80): tipo resposta dada ao recurso (campo em branco para recursos que ainda estejam na situação "Em Tramitação");

------------------------------
-- CAMPOS: SOLICITANTES
------------------------------

- IdSolicitante - inteiro: identificador único do solicitante (não mostrado no sistema);
- TipoDemandante - texto(15): Pessoa Fìsica ou Pessoa Jurídica;
- DataNascimento - Data DD/MM/AAAA : data de nascimento do solicitante;
- Genero - texto(13) : Masculino, Feminino ou Outro(em branco para pessoa jurídica);
- Escolaridade - texto(200): Escolaridade do solicitante (em branco para pessoa jurídica);
- Profissao - texto(200): Profissão do solicitante (em branco para pessoa jurídica);
- TipoPessoaJuridica - texto(200): tipo de Pessoa Jurídica do solicitante (em branco para pessoa física)
- Pais - texto(200): país de residência do solicitante;
- UF - texto(2): UF de residência do solicitante;
- Municipio - texto(200): Município de residência do solicitante;

In [1]:
import requests
from datetime import date
import shutil
import os
import time
import zipfile
import pandas as pd
import re
from wordcloud import WordCloud
from nltk.corpus import stopwords
from string import punctuation
import matplotlib.pyplot as plt


In [2]:
header_pedidos = ['IdPedido',
                'ProtocoloPedido',
                'Esfera',
                'UF',
                'Municipio',
                'OrgaoDestinatario',
                'Situacao',
                'DataRegistro',
                'PrazoAtendimento',
                'FoiProrrogado',
                'FoiReencaminhado',
                'FormaResposta',
                'OrigemSolicitacao',
                'IdSolicitante',
                'AssuntoPedido',
                'SubAssuntoPedido',
                'Tag',
                'DataResposta',
                'Decisao',
                'EspecificacaoDecisao']

header_recursos = ['IdRecurso',
                'IdRecursoPrecedente',
                'IdPedido',
                'IdSolicitante',
                'ProtocoloPedido',
                'Esfera',
                'UF',
                'Municipio',
                'OrgaoDestinatario',
                'Instancia',
                'Situacao',
                'DataRegistro',
                'PrazoAtendimento',
                'OrigemSolicitacao',
                'TipoRecurso',
                'DataResposta',
                'TipoResposta']

header_solicitante = ['IdSolicitante',
                    'TipoDemandante',
                    'DataNascimento',
                    'Genero',
                    'Escolaridade',
                    'Profissao',
                    'TipoPessoaJuridica',
                    'Pais',
                    'UF',
                    'Municipio']

In [3]:
ANO_INICIO=2012
PREFIXO_ACESSO = "Pedidos"
PREFIXO_RECURSO = "Recursos_Reclamacoes"
NOME_ARQUIVO_BASE = "{prefixo}_{formato_arquivo}_{ano}.zip"
URL_BASE = "https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR/{nome_arquivo}"

NOME_ARQUIVO_FILTRADO = 'Arquivos_{formato_arquivo}_{ano}.zip'
URL_FILTRADOS = "https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR_Filtrado/{nome_arquivo}"


In [6]:
def busca_lai_por_anos(anos, tipo_dados='acesso', localizacao='./', formato_arquivo='csv'):
    """
    anos: Lista de anos para download.
    tipo_dados: Se o arquivo é de acesso ou recurso
    localizacao: Onde salvar os arquivos
    formato_arquivo: formatos xml|csv
    """
    if tipo_dados.lower() == "acesso":
        prefix = PREFIXO_ACESSO
    else:
        prefix = PREFIXO_RECURSO

    for year in anos:
        nome_arquivo  = NOME_ARQUIVO_BASE.format(prefixo=prefix, formato_arquivo=formato_arquivo, ano=str(year))
        url = URL_BASE.format(nome_arquivo=nome_arquivo)
        try:
            response = requests.get(url, stream=True)
            with open(os.path.join(localizacao, nome_arquivo), 'wb') as out_file:
                shutil.copyfileobj(response.raw, out_file)

            print(f"Arquivo {nome_arquivo} baixado de {url}")
        except Exception as e:
            print(e)
            print(f"Error baixando arquivo {nome_arquivo} da url {url}")

        time.sleep(2)
        
def unzip_arquivos(de='zips', diretorio_destino='csvs'):
    #https://stackoverflow.com/questions/3451111/unzipping-files-in-python

    pattern = re.compile(".*\.zip")
    files = os.listdir(de)
    for file in files:
        if os.path.isfile(os.path.join(de, file)) and pattern.match(file.lower()) != None:    
            arquivo = os.path.join(de, file)
            with zipfile.ZipFile(arquivo,"r") as arquivo_zip:
                print(f'descompactando {arquivo}')
                arquivo_zip.extractall(diretorio_destino)
        

def busca_textos_por_anos(anos, localizacao='./', formato_arquivo='csv'):
    """
    anos: Lista de anos para download.
    localizacao: Onde salvar os arquivos
    formato_arquivo: formatos xml|csv
    """
    
    for year in anos:
        nome_arquivo  = NOME_ARQUIVO_FILTRADO.format(formato_arquivo=formato_arquivo, ano=str(year))
        url = URL_FILTRADOS.format(nome_arquivo=nome_arquivo)
        try:
            response = requests.get(url, stream=True)
            with open(os.path.join(localizacao, nome_arquivo), 'wb') as out_file:
                shutil.copyfileobj(response.raw, out_file)

            print(f"Arquivo {nome_arquivo} baixado de {url}")
        except Exception as e:
            print(e)
            print(f"Error baixando arquivo {nome_arquivo} da url {url}")

        time.sleep(2)
        
        
class Pedidos:
    
    @staticmethod
    def carrega_arquivos_csv_df(diretorio, header_pedidos, parse_dates=None):
        pattern = re.compile(".*\_pedidos\_.*\.csv")
        df = pd.DataFrame()
        files = os.listdir(diretorio)
        if parse_dates is None:
            parse_dates=['DataRegistro', 'PrazoAtendimento', 'DataResposta']
            
        for file in files:
            if os.path.isfile(os.path.join(diretorio, file)) and pattern.match(file.lower()) != None:
                print(f"Carregando {file}")
                df = pd.concat([df, pd.read_csv(os.path.join(diretorio, file), names=header_pedidos, sep=';', encoding='utf-16', parse_dates=parse_dates, dayfirst=True, engine='c')], axis=0)
                print(f'Carregado, memória utilizada após carga: {round(df.memory_usage(deep=True).sum()/(1024*1024), 2)}MB')
        return df

    @staticmethod
    def carrega_arquivos_xml_df(diretorio, header_pedidos, parse_dates):
        pattern = re.compile(".*\_pedidos\_.*\.xml")
        df = pd.DataFrame()
        files = os.listdir(diretorio)
        for file in files:
            if os.path.isfile(os.path.join(diretorio, file)) and pattern.match(file.lower()) != None:
                print(f"Carregando {file}")
                df = pd.concat([df, pd.read_xml(os.path.join(diretorio, file), names=header_pedidos, encoding='utf-16', parse_dates=parse_dates, dayfirst=True, engine='c')])
                print(f'Carregado, memória utilizada após carga: {round(df.memory_usage(deep=True).sum()/(1024*1024), 2)}MB')
        return df


class Recursos:
    
    @staticmethod
    def carrega_arquivos_csv_df(diretorio, header_recursos, parse_dates=None):
        pattern = re.compile(".*\_recursos\_.*\.csv")
        df = pd.DataFrame()
        files = os.listdir(diretorio)
        if parse_dates is None:
            parse_dates=['DataRegistro', 'PrazoAtendimento', 'DataResposta']
            
        for file in files:
            if os.path.isfile(os.path.join(diretorio, file)) and pattern.match(file.lower()) != None:
                print(f"Carregando {file}")
                df = pd.concat([df, pd.read_csv(os.path.join(diretorio, file), names=header_recursos, sep=';', encoding='utf-16', parse_dates=parse_dates, dayfirst=True, engine='c')], axis=0)
                print(f'Carregado, memória utilizada após carga: {round(df.memory_usage(deep=True).sum()/(1024*1024), 2)}MB')
        return df

    @staticmethod
    def carrega_arquivos_xml_df(diretorio, header_recursos, parse_dates):
        pattern = re.compile(".*\_recursos\_.*\.xml")
        df = pd.DataFrame()
        files = os.listdir(diretorio)
        for file in files:
            if os.path.isfile(os.path.join(diretorio, file)) and pattern.match(file.lower()) != None:
                print(f"Carregando {file}")
                df = pd.concat([df, pd.read_xml(os.path.join(diretorio, file), names=header_recursos, encoding='utf-16', parse_dates=parse_dates, dayfirst=True, engine='c')])
                print(f'Carregado, memória utilizada após carga: {round(df.memory_usage(deep=True).sum()/(1024*1024), 2)}MB')
        return df

In [5]:
anos = range(ANO_INICIO, int(date.today().strftime("%Y"))+1)
list(anos)


[2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

In [6]:
busca_lai_por_anos(anos, 'acesso', 'zips_lai_rel', 'csv')


Arquivo Pedidos_csv_2012.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR/Pedidos_csv_2012.zip
Arquivo Pedidos_csv_2013.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR/Pedidos_csv_2013.zip
Arquivo Pedidos_csv_2014.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR/Pedidos_csv_2014.zip
Arquivo Pedidos_csv_2015.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR/Pedidos_csv_2015.zip
Arquivo Pedidos_csv_2016.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR/Pedidos_csv_2016.zip
Arquivo Pedidos_csv_2017.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR/Pedidos_csv_2017.zip
Arquivo Pedidos_csv_2018.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR/Pedidos_csv_2018.zip
Arquivo Pedidos_csv_2019.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR/Pedidos_csv_2019.zip


In [7]:
busca_lai_por_anos(anos, 'recurso', 'zips_lai_rel', 'csv')


Arquivo Recursos_Reclamacoes_csv_2012.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR/Recursos_Reclamacoes_csv_2012.zip
Arquivo Recursos_Reclamacoes_csv_2013.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR/Recursos_Reclamacoes_csv_2013.zip
Arquivo Recursos_Reclamacoes_csv_2014.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR/Recursos_Reclamacoes_csv_2014.zip
Arquivo Recursos_Reclamacoes_csv_2015.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR/Recursos_Reclamacoes_csv_2015.zip
Arquivo Recursos_Reclamacoes_csv_2016.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR/Recursos_Reclamacoes_csv_2016.zip
Arquivo Recursos_Reclamacoes_csv_2017.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR/Recursos_Reclamacoes_csv_2017.zip
Arquivo Recursos_Reclamacoes_csv_2018.zip baixado de https://dadosabertos-download.cgu.gov.br/

In [8]:
unzip_arquivos(de='zips_lai_rel', diretorio_destino='csvs_rel')
               

descompactando zips_lai_rel/Pedidos_csv_2023.zip
descompactando zips_lai_rel/Recursos_Reclamacoes_csv_2016.zip
descompactando zips_lai_rel/Recursos_Reclamacoes_csv_2015.zip
descompactando zips_lai_rel/Pedidos_csv_2021.zip
descompactando zips_lai_rel/Recursos_Reclamacoes_csv_2013.zip
descompactando zips_lai_rel/Pedidos_csv_2016.zip
descompactando zips_lai_rel/Pedidos_csv_2015.zip
descompactando zips_lai_rel/Recursos_Reclamacoes_csv_2014.zip
descompactando zips_lai_rel/Recursos_Reclamacoes_csv_2017.zip
descompactando zips_lai_rel/Pedidos_csv_2020.zip
descompactando zips_lai_rel/Pedidos_csv_2019.zip
descompactando zips_lai_rel/Recursos_Reclamacoes_csv_2020.zip
descompactando zips_lai_rel/Recursos_Reclamacoes_csv_2021.zip
descompactando zips_lai_rel/Pedidos_csv_2013.zip
descompactando zips_lai_rel/Pedidos_csv_2012.zip
descompactando zips_lai_rel/Pedidos_csv_2014.zip
descompactando zips_lai_rel/Recursos_Reclamacoes_csv_2018.zip
descompactando zips_lai_rel/Recursos_Reclamacoes_csv_2023.zip
d

In [16]:
%%time
df = Pedidos.carrega_arquivos_csv_df('csvs_rel', header_pedidos=header_pedidos)


Carregando 20230302_Pedidos_csv_2012.csv
Carregado, memória utilizada após carga: 64.37MB
Carregando 20230307_Pedidos_csv_2017.csv
Carregado, memória utilizada após carga: 203.24MB
Carregando 20230307_Pedidos_csv_2022.csv
Carregado, memória utilizada após carga: 365.42MB
Carregando 20230307_Pedidos_csv_2013.csv
Carregado, memória utilizada após carga: 475.16MB
Carregando 20230302_Pedidos_csv_2017.csv
Carregado, memória utilizada após carga: 626.51MB
Carregando 20230302_Pedidos_csv_2021.csv
Carregado, memória utilizada após carga: 775.61MB
Carregando 20230302_Pedidos_csv_2022.csv
Carregado, memória utilizada após carga: 918.99MB
Carregando 20230302_Pedidos_csv_2019.csv
Carregado, memória utilizada após carga: 1080.98MB
Carregando 20230307_Pedidos_csv_2019.csv
Carregado, memória utilizada após carga: 1242.98MB
Carregando 20230307_Pedidos_csv_2016.csv
Carregado, memória utilizada após carga: 1382.62MB
Carregando 20230302_Pedidos_csv_2014.csv
Carregado, memória utilizada após carga: 1491.4

In [17]:
df['DataRegistro'].info

<bound method Series.info of 0        2012-05-15
1        2012-05-15
2        2012-05-15
3        2012-05-15
4        2012-05-15
            ...    
120785   2017-12-31
120786   2017-12-31
120787   2017-12-31
120788   2017-12-31
120789   2017-12-31
Name: DataRegistro, Length: 3626677, dtype: datetime64[ns]>

In [18]:
df['AnoRegistro'] = df['DataRegistro'].dt.year


In [19]:
df.to_pickle('dados/pedidos_relatorio.pkl')


In [7]:
%%time
df = Recursos.carrega_arquivos_csv_df('csvs_rel', header_recursos=header_recursos)


Carregando 20230302_Recursos_Reclamacoes_csv_2014.csv
Carregado, memória utilizada após carga: 9.3MB
Carregando 20230307_Recursos_Reclamacoes_csv_2016.csv
Carregado, memória utilizada após carga: 24.05MB
Carregando 20230307_Recursos_Reclamacoes_csv_2019.csv
Carregado, memória utilizada após carga: 41.37MB
Carregando 20230302_Recursos_Reclamacoes_csv_2013.csv
Carregado, memória utilizada após carga: 51.25MB
Carregando 20230302_Recursos_Reclamacoes_csv_2020.csv
Carregado, memória utilizada após carga: 70.04MB
Carregando 20230310_Recursos_Reclamacoes_csv_2016.csv
Carregado, memória utilizada após carga: 84.71MB
Carregando 20230310_Recursos_Reclamacoes_csv_2022.csv
Carregado, memória utilizada após carga: 99.4MB
Carregando 20230307_Recursos_Reclamacoes_csv_2017.csv
Carregado, memória utilizada após carga: 118.65MB
Carregando 20230307_Recursos_Reclamacoes_csv_2013.csv
Carregado, memória utilizada após carga: 128.53MB
Carregando 20230307_Recursos_Reclamacoes_csv_2021.csv
Carregado, memória u

In [8]:
df['AnoRegistro'] = df['DataRegistro'].dt.year


In [9]:
df.to_pickle('dados/recursos_relatorio.pkl')


## Arquivos com textos de pedidos

No menu localizado no final da página existe um link para a Busca de Pedidos e Respostas. Nesse link pode-se encontrar a opção de download de dados.

![tela fala br download com texto](imagens/falabr_tela_download_busca.png "Menu do Fala.Br. Busca de Pedidos e Respostas")

![tela fala br download](imagens/falabr_tela_download_busca_select.png "Tela do Fala.Br para download dos arquivos de Pedidos e Recursos sem o texto do pedido.")

------------------------------
-- CAMPOS: PEDIDOS
------------------------------
- IdPedido - inteiro: identificador único do pedido (não mostrado no sistema);
- ProtocoloPedido - texto(17): número do protocolo do pedido;
- Esfera - texto(30): esfera do órgão destinatário do pedido; 
- OrgaoDestinatario  - texto(250): nome do órgão destinatário do pedido;
- Situacao - texto(200): descrição da situação do pedido;
- DataRegistro - Data DD/MM/AAAA HH:MM:SS : data de abertura do pedido;
- ResumoSolicitacao - texto(255): resumo do pedido;
- DetalhamentoSolicitacao - texto(2048): detalhamento do pedido;
- PrazoAtendimento - Data DD/MM/AAAA HH:MM:ss : data limite para atendimento ao pedido;
- FoiProrrogado - texto(3) "Sim" ou "Não" : informa se houve prorrogação do prazo do pedido;
- FoiReencaminhado - texto(3) "Sim" ou "Não": informa se o pedido foi reencaminhado;
- FormaResposta - texto(200): tipo de resposta escolhida pelo solicitante na abertura do pedido;
- OrigemSolicitacao - texto(50): informa se o pedido foi aberto em um Balcão SIC ou pela Internet;
- IdSolicitante - inteiro: identificador único do solicitante (não mostrado no sistema);
- AssuntoPedido - texto(200) : assunto do pedido atribuído pel SIC;
- SubAssuntoPedido - texto(200) : subassunto do pedido atribuída pelo SIC;
- Tag - texto(1024): as tags são marcadores no pedido para realizar classificações que não estão em assuntos/ subassuntos;
- DataResposta - Data DD/MM/AAAA HH:MM:SS : data da resposta ao pedido (campo em branco para pedidos que ainda estejam na situação "Em Tramitação");
- Resposta - texto(8000): resposta ao pedido;
- Decisao - texto(100) : tipo resposta dada ao pedido (campo em branco para pedidos que ainda estejam na situação "Em Tramitação");
- EspecificacaoDecisao - texto(200): subtipo da resposta dada ao pedido (campo em branco para pedidos que ainda estejam na situação "Em Tramitação");


------------------------------
-- CAMPOS: RECURSOS
------------------------------

- IdRecurso - inteiro: identificador único do recurso (não mostrado no sistema);
- IdRecursoPrecedente - inteiro: identificador único do recurso que precedeu este (não mostrado no sistema e em branco no caso de Recursos de 1ª Instância e Reclamações);
- DescRecurso - texto(8000): descrição do recurso;
- IdPedido - inteiro: identificador único do pedido ao qual o recurso pertence (não mostrado no sistema);
- IdSolicitante - inteiro: identificador único do solicitante (não mostrado no sistema);
- ProtocoloPedido - texto(17): número do protocolo do pedido ao qual o recurso pertence;
- OrgaoDestinatario - texto(250): nome do órgão destinatário do recurso;
- Instancia - texto(80): descrição da instância do recurso;
- Situacao - texto(80): descrição da situação do recurso;
- DataRegistro - Data DD/MM/AAAA HH:MM:SS : data de abertura do recurso;
- PrazoAtendimento - Data DD/MM/AAAA HH:MM:SS : data limite para atendimento ao recurso;
- OrigemSolicitacao - texto(50): informa se o recurso foi aberto em um Balcão SIC ou pela Internet;
- TipoRecurso - texto(80): motivo de abertura do recurso;
- DataResposta - Data DD/MM/AAAA HH:MM:SS : data da resposta ao recurso (campo em branco para recursos que ainda estejam na situação "Em Tramitação");
- RespostaRecurso - texto(8000): resposta ao recurso;
- TipoResposta - texto(80): tipo resposta dada ao recurso (campo em branco para recursos que ainda estejam na situação "Em Tramitação");

 
------------------------------
-- CAMPOS: SOLICITANTES
------------------------------

- IdSolicitante - inteiro: identificador único do solicitante (não mostrado no sistema);
- TipoDemandante - texto(15): Pessoa Fìsica ou Pessoa Jurídica;
- DataNascimento - Data DD/MM/AAAA : data de nascimento do solicitante;
- Genero - texto(13) : Masculino, Feminino ou Outro(em branco para pessoa jurídica);
- Escolaridade - texto(200): Escolaridade do solicitante (em branco para pessoa jurídica);
- Profissao - texto(200): Profissão do solicitante (em branco para pessoa jurídica);
- TipoPessoaJuridica - texto(200): tipo de Pessoa Jurídica do solicitante (em branco para pessoa física)
- Pais - texto(200): país de residência do solicitante;
- UF - texto(2): UF de residência do solicitante;
- Municipio - texto(200): Município de residência do solicitante;

In [11]:
pedido_header_txts = ['IdPedido',
                'ProtocoloPedido',
                'Esfera',
                'OrgaoDestinatario',
                'Situacao',
                'DataRegistro',
                'ResumoSolicitacao',
                'DetalhamentoSolicitacao',
                'PrazoAtendimento',
                'FoiProrrogado',
                'FoiReencaminhado',
                'FormaResposta',
                'OrigemSolicitacao',
                'IdSolicitante',
                'AssuntoPedido',
                'SubAssuntoPedido',
                'Tag',
                'DataResposta',
                'Resposta',
                'Decisao',
                'EspecificacaoDecisao']

recurso_header_txts = ['IdRecurso',
                'IdRecursoPrecedente',
                'DescRecurso',
                'IdPedido',
                'IdSolicitante',
                'ProtocoloPedido',
                'OrgaoDestinatario',
                'Instancia',
                'Situacao',
                'DataRegistro',
                'PrazoAtendimento',
                'OrigemSolicitacao',
                'TipoRecurso',
                'DataResposta',
                'RespostaRecurso',
                'TipoResposta']


solicitante_header_txts = ['IdSolicitante',
                'TipoDemandante',
                'DataNascimento',
                'Genero',
                'Escolaridade',
                'Profissao',
                'TipoPessoaJuridica',
                'Pais',
                'UF',
                'Municipio']


In [21]:
anos = range(2015, int(date.today().strftime("%Y"))+1)
list(anos)


[2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

In [22]:
busca_textos_por_anos(anos, localizacao='zips_lai_txts', formato_arquivo='csv')


Arquivo Arquivos_csv_2015.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR_Filtrado/Arquivos_csv_2015.zip
Arquivo Arquivos_csv_2016.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR_Filtrado/Arquivos_csv_2016.zip
Arquivo Arquivos_csv_2017.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR_Filtrado/Arquivos_csv_2017.zip
Arquivo Arquivos_csv_2018.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR_Filtrado/Arquivos_csv_2018.zip
Arquivo Arquivos_csv_2019.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR_Filtrado/Arquivos_csv_2019.zip
Arquivo Arquivos_csv_2020.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR_Filtrado/Arquivos_csv_2020.zip
Arquivo Arquivos_csv_2021.zip baixado de https://dadosabertos-download.cgu.gov.br/FalaBR/Arquivos_FalaBR_Filtrado/Arquivos_csv_2021.zip
Arquivo Arquivos_csv_2022.zip baixado de https:/

In [23]:
unzip_arquivos(de='zips_lai_txts', diretorio_destino='csvs_txts')


descompactando zips_lai_txts/Arquivos_csv_2018.zip
descompactando zips_lai_txts/Arquivos_csv_2023.zip
descompactando zips_lai_txts/Arquivos_csv_2021.zip
descompactando zips_lai_txts/Arquivos_csv_2016.zip
descompactando zips_lai_txts/Arquivos_csv_2020.zip
descompactando zips_lai_txts/Arquivos_csv_2019.zip
descompactando zips_lai_txts/Arquivos_csv_2022.zip
descompactando zips_lai_txts/Arquivos_csv_2015.zip
descompactando zips_lai_txts/Arquivos_csv_2017.zip


In [16]:
%%time
df = Pedidos.carrega_arquivos_csv_df('csvs_txts', header_pedidos=pedido_header_txts)

Carregando 20230310_Pedidos_csv_2016.csv
Carregado, memória utilizada após carga: 252.54MB
Carregando 20230310_Pedidos_csv_2020.csv
Carregado, memória utilizada após carga: 674.73MB
Carregando 20230310_Pedidos_csv_2022.csv
Carregado, memória utilizada após carga: 931.24MB
Carregando 20230310_Pedidos_csv_2019.csv
Carregado, memória utilizada após carga: 1246.86MB
Carregando 20230310_Pedidos_csv_2018.csv
Carregado, memória utilizada após carga: 1533.7MB
Carregando 20230310_Pedidos_csv_2021.csv
Carregado, memória utilizada após carga: 1834.95MB
Carregando 20230310_Pedidos_csv_2023.csv
Carregado, memória utilizada após carga: 1881.18MB
Carregando 20230310_Pedidos_csv_2015.csv
Carregado, memória utilizada após carga: 2101.27MB
Carregando 20230310_Pedidos_csv_2017.csv
Carregado, memória utilizada após carga: 2368.27MB
CPU times: user 19.5 s, sys: 16.1 s, total: 35.7 s
Wall time: 35.7 s


In [17]:
df.head()

,IdPedido,ProtocoloPedido,Esfera,OrgaoDestinatario,Situacao,DataRegistro,ResumoSolicitacao,DetalhamentoSolicitacao,PrazoAtendimento,FoiProrrogado,...,FormaResposta,OrigemSolicitacao,IdSolicitante,AssuntoPedido,SubAssuntoPedido,Tag,DataResposta,Resposta,Decisao,EspecificacaoDecisao
0,1678719,99901000001201633,Federal,BB – Banco do Brasil S.A.,Concluída,2016-01-01,consultar meu cpf . no spc e serasa,Consulta do cpf no serasa e spc,2016-01-25,Não,...,Pelo sistema (com avisos por email),Internet,2625397,Outros em Economia e Finanças,,,2016-01-25,"Prezado(a) Sr(a). Gisela, Encaminhamos, em an...",Acesso Concedido,Resposta solicitada inserida no Fala.Br
1,1678723,99902000001201623,Federal,CEF – Caixa Econômica Federal,Concluída,2016-01-01,Esclarecimentos sobre seguro de risco de incêndio,Sou conselheiro de um Condomínio e depois da p...,2016-01-25,Não,...,Pelo sistema (com avisos por email),Internet,2571729,Outros em Economia e Finanças,,,2016-01-05,"Prezado (a) Senhor (a), 1. Em atenção a su...",Acesso Negado,Pedido incompreensível
2,1678726,99901000002201688,Federal,BB – Banco do Brasil S.A.,Concluída,2016-01-01,SALDO PASEP,"Bom dia, necessito do saldo do PASEP",2016-01-25,Não,...,Pelo sistema (com avisos por email),Internet,2117083,Outros em Economia e Finanças,,,2016-01-25,"Prezado(a) Sr(a). Regina, Encaminhamos, em an...",Acesso Concedido,Resposta solicitada inserida no Fala.Br
3,1678739,80200000001201611,Federal,MTur - Ministério do Turismo,Concluída,2016-01-01,cópia termo de convenio em MCIDADES e prefeitu...,Solicito cópia dos termos de convenio nº siafi...,2016-01-25,Não,...,Pelo sistema (com avisos por email),Internet,47991,Turismo,,,2016-01-05,Segue resposta da área técnica Retransmitimo...,Acesso Concedido,Resposta solicitada inserida no Fala.Br
4,1678747,80200000002201657,Federal,MDR - Ministério do Desenvolvimento Regional,Concluída,2016-01-01,cópia termo de convenio em MCIDADES e prefeitu...,Solicito copia do termo de convenio nº siafi 5...,2016-01-25,Não,...,Pelo sistema (com avisos por email),Internet,47991,Infraestrutura Urbana,,,2016-01-04,"Prezado Sr. Maxoel, Por essa demanda se trata...",Pergunta Duplicada/Repetida,


In [18]:
df['AnoRegistro'] = df['DataRegistro'].dt.year

In [19]:
df.to_pickle('dados/pedidos_texto.pkl')

In [20]:
%%time
df = Recursos.carrega_arquivos_csv_df('csvs_txts', header_recursos=recurso_header_txts)


Carregando 20230310_Recursos_csv_2023.csv
Carregado, memória utilizada após carga: 6.08MB
Carregando 20230310_Recursos_csv_2019.csv
Carregado, memória utilizada após carga: 53.02MB
Carregando 20230310_Recursos_csv_2021.csv
Carregado, memória utilizada após carga: 95.19MB
Carregando 20230310_Recursos_csv_2016.csv
Carregado, memória utilizada após carga: 130.07MB
Carregando 20230310_Recursos_csv_2022.csv
Carregado, memória utilizada após carga: 169.43MB
Carregando 20230310_Recursos_csv_2018.csv
Carregado, memória utilizada após carga: 212.29MB
Carregando 20230310_Recursos_csv_2020.csv
Carregado, memória utilizada após carga: 265.54MB
Carregando 20230310_Recursos_csv_2017.csv
Carregado, memória utilizada após carga: 306.63MB
Carregando 20230310_Recursos_csv_2015.csv
Carregado, memória utilizada após carga: 336.13MB
CPU times: user 2.47 s, sys: 812 ms, total: 3.28 s
Wall time: 3.28 s


In [21]:
df['AnoRegistro'] = df['DataRegistro'].dt.year

In [22]:
df.to_pickle('dados/recursos_texto.pkl')